# <center>Colorado K-12 Financial Transparency – <br>An Exploratory Data Science Comparison Across Schools</center>

### <center>MSDS 696: Data Science Practicum II - Stacey Sandy</center>

### <u><b>Overview </b></u>
This project is for the fulfillment of my final course in the pursuit of my Master of Science in Data Science degree at Regis University. In this past academic year, my kindergartner son has brought home 5 different fundraisers and a quarterly scholastic book fair flyer. I understand the scholastic book fair awards books to classes that purchase the most books. Unfortunately, I am unclear where the multiple fundraiser funding is going, what the funding is for, and how fundraisers feed into the school. These implications on the school and district fundraiser efforts need further analysis. Especially since school ends in a little over two months, I am curious to learn how expenses are handled. I intend to compare expenses across schools and then across districts. I would like to analyze which schools are spending more than others and why their expenses are more or less than others. I want to learn the cause and effects as to what are the trends on expenditures. Hopefully, I could learn more about why fundraising is important or necessary multiple times a year for any school. 

### <u><b>Objective</b></u>
The objective of this project and the primary research question is to address how financial expenditures are being spent across K through 12 schools in Colorado. An exploratory data science approach will be conducted to answer this research question. Overall, what are the school expenses that impact our children’s educational needs? Could the type of staffing, teacher types, and number of students be an impact? What are the contributing factors of the various expenses? How do schools and districts compare to one another? What variables are impacting the data reported? Could federal and state funding not be enough? 

### <u><b>Data Source</b></u>
* Colorado Department of Education website on Colorado’s K12 Financial Transparency website: https://coloradok12financialtransparency.com/#/
* The complete dataset of all Colorado K12 districts and BOCES can also be downloaded at: https://s3.amazonaws.com/bb-pub-pipeline-production/home/circleci/repo/public/uploads/data_files/uploads/000/000/416/20200114030354-colorado-financial-public-data-pipeline-v-1-Run_1-export-source-lep-facts.csv20200114-2678-rqjqmn.csv?1578999834

### <u><b>Financial Policies and Procedures Documentation</b></u>
* Colorado Department of Education. (2016, July 1). <i>Financial Policies and Procedures Chart of Accounts.</i> Retrieved from https://www.cde.state.co.us/cdefinance/fpp_coa1617

#### The following website is available for direct school or district searchs within Colorado:
* Colorado K12 Financial Transparency website: https://coloradok12financialtransparency.com/#/

### Project Process:<br>
<br>
1. Conduct exploratory data analysis on complete dataset of all Colorado K12 districts and BOCES.<br>
2. Investigate unique Ids, values, and row/column data.<br>
3. <br>
4. <br>
5. <br.
<br>
In final, compare and contrast findings. Answer research questions. Did you accomplish what your project objective detailed?

## <center>Exploratory Data Analysis (EDA) on all Colorado K12 districts and BOCES</center>

In [9]:
# Load libraries
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
import multiprocessing
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import requests
import json

In [2]:
# Load the CO K12 Dataset into a dataframe.
COK12_df = pd.read_csv('Data/colorado-financial-public-data-pipeline-v-1-Run#1-export-source-lep-facts.csv', header=4)

In [3]:
# Ensure columns were retrieved
COK12_df.columns

Index(['district_code', 'admin_unit', 'school_code', 'fund_code',
       'location_code', 'sre_code', 'program_code', 'object_source_code',
       'job_class_code', 'grant_code', 'amount_cents', 'exclude_school',
       'exclude_district'],
      dtype='object')

In [4]:
# Look at dataset head
COK12_df.head()

,district_code,admin_unit,school_code,fund_code,location_code,sre_code,program_code,object_source_code,job_class_code,grant_code,amount_cents,exclude_school,exclude_district
0,3080,64203,0,10,600,0,20,100,200,0,71250,f,f
1,3080,64203,0,10,600,0,10,100,200,4010,13841274,f,f
2,3080,64203,0,10,700,0,2700,100,100,0,12171639,f,f
3,3080,64203,0,21,700,0,3100,100,100,0,4079700,f,f
4,3080,64203,4854,10,200,0,1700,100,400,3130,8660925,f,f


In [5]:
# Look at dataset tail
COK12_df.tail()

,district_code,admin_unit,school_code,fund_code,location_code,sre_code,program_code,object_source_code,job_class_code,grant_code,amount_cents,exclude_school,exclude_district
280681,1180,64233,570,10,300,0,1800,810,0,0,340700,f,f
280682,1180,64233,429,11,951,0,2500,810,0,0,-700,f,f
280683,1180,64233,429,11,951,0,2600,810,0,0,45477,f,f
280684,1180,64233,429,11,951,0,2800,810,0,0,24500,f,f
280685,1180,64233,429,11,951,0,18,810,0,0,2499,f,f


In [6]:
# Look at dataset information details
COK12_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280686 entries, 0 to 280685
Data columns (total 13 columns):
district_code         280686 non-null int64
admin_unit            280686 non-null int64
school_code           280686 non-null int64
fund_code             280686 non-null int64
location_code         280686 non-null int64
sre_code              280686 non-null int64
program_code          280686 non-null int64
object_source_code    280686 non-null int64
job_class_code        280686 non-null int64
grant_code            280686 non-null int64
amount_cents          280686 non-null int64
exclude_school        280686 non-null object
exclude_district      280686 non-null object
dtypes: int64(11), object(2)
memory usage: 27.8+ MB


In [18]:
# Generate pandas profile report
COK12_profile = ProfileReport(COK12_df, title='Pandas CO K12 Profiling Report', html={'style':{'full_width':True}})

TypeError: _plot_histogram() got an unexpected keyword argument 'title'

In [11]:
if name=="main":
    multiprocessing.freeze_support()
create_pd_profile()

NameError: name 'name' is not defined

In [ ]:
# Display profile report in Jupyter Notebook widgets
COK12_profile.to_widgets()

In [ ]:
# Include html report in Jupyter Noteboook
COK12_profile.to_notebook_iframe()

In [ ]:
# Save the profile report out to html
COK12_profile.to_file(output_file="your_report.html")